In [6]:
from genai import Client, Credentials

client = Client(credentials=Credentials.from_env())

In [7]:
import os

os.environ["GENAI_KEY"] = "pak-8prwoIymlhiNFNhxydsvEL8pl2VnQi_84sy03fvyLC0"
os.environ["GENAI_API"] = "https://bam-api.res.ibm.com/"

from genai.credentials import Credentials

credentials = Credentials.from_env()
client = Client(credentials=Credentials.from_env())

In [9]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage

from genai import Client, Credentials
from genai.extensions.langchain.chat_llm import LangChainChatInterface
from genai.schema import (
    DecodingMethod,
    ModerationHAP,
    ModerationHAPInput,
    ModerationParameters,
    TextGenerationParameters,
    TextGenerationReturnOptions,
)

# make sure you have a .env file under genai root with
# GENAI_KEY=<your-genai-key>
# GENAI_API=<genai-api-endpoint> (optional) DEFAULT_API = "https://bam-api.res.ibm.com"
load_dotenv()


def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"


llm = LangChainChatInterface(
    client=Client(credentials=Credentials.from_env()),
    model_id="meta-llama/llama-2-70b-chat",
    parameters=TextGenerationParameters(
        decoding_method=DecodingMethod.SAMPLE,
        max_new_tokens=100,
        min_new_tokens=10,
        temperature=0.5,
        top_k=50,
        top_p=1,
        return_options=TextGenerationReturnOptions(input_text=False, input_tokens=True),
    ),
    moderations=ModerationParameters(
        # Threshold is set to very low level to flag everything (testing purposes)
        # or set to True to enable HAP with default settings
        hap=ModerationHAP(input=ModerationHAPInput(enabled=True, threshold=0.01))
    ),
)

print(heading("Start conversation with langchain"))
prompt = input("Enter your prompt: ")
print(f"Request: {prompt}")
result = llm.generate(
    messages=[
        [
            SystemMessage(
                content="""You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature. If a question does not make
any sense, or is not factually coherent, explain why instead of answering something incorrectly.
If you don't know the answer to a question, please don't share false information.
""",
            ),
            HumanMessage(content=prompt),
        ]
    ],
)
conversation_id = result.generations[0][0].generation_info["meta"]["conversation_id"]
print(f"New conversation with ID '{conversation_id}' has been created!")
print(f"Response: {result.generations[0][0].text}")
print(result.llm_output)
print(result.generations[0][0].generation_info)

print(heading("Continue conversation with langchain"))
prompt = input("Enter your next prompt: ")
print(f"Request: {prompt}")
result = llm.generate(
    messages=[[HumanMessage(content=prompt)]], conversation_id=conversation_id, use_conversation_parameters=True
)
print(f"Response: {result.generations[0][0].text}")


====================== Start conversation with langchain =======================

Request: Hi, My name is Saima
New conversation with ID '9debc536-f412-462e-9698-d59e7b63e294' has been created!
Response:   Hello Saima! It's nice to meet you. Is there anything I can assist you with today? Please feel free to ask me any questions, and I'll do my best to provide you with helpful and accurate information. Remember, I'm here to help and support you in any way I can, while ensuring that my responses are respectful, safe, and socially unbiased.
{'model_name': 'meta-llama/llama-2-70b-chat', 'token_usage': {'prompt_tokens': 148, 'completion_tokens': 85, 'total_tokens': 233}}
{'meta': {'conversation_id': '9debc536-f412-462e-9698-d59e7b63e294', 'created_at': datetime.datetime(2024, 5, 6, 11, 26, 52, 110000, tzinfo=TzInfo(UTC)), 'id': 'ca29b2cb-27d4-47ab-80e1-2257e5722132', 'model_id': 'meta-llama/llama-2-70b-chat'}, 'token_usage': {'prompt_tokens': 148, 'completion_tokens': 85, 'total_tokens': 2

## Running in loop

In [12]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage

from genai import Client, Credentials
from genai.extensions.langchain.chat_llm import LangChainChatInterface
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from genai.schema import (
    DecodingMethod,
    ModerationHAP,
    ModerationHAPInput,
    ModerationParameters,
    TextGenerationParameters,
    TextGenerationReturnOptions,
)

# make sure you have a .env file under genai root with
# GENAI_KEY=<your-genai-key>
# GENAI_API=<genai-api-endpoint> (optional) DEFAULT_API = "https://bam-api.res.ibm.com"
load_dotenv()


def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"


llm = LangChainChatInterface(
    client=Client(credentials=Credentials.from_env()),
    model_id="meta-llama/llama-2-70b-chat",
    parameters=TextGenerationParameters(
        decoding_method=DecodingMethod.SAMPLE,
        max_new_tokens=100,
        min_new_tokens=10,
        temperature=0.5,
        top_k=50,
        top_p=1,
        return_options=TextGenerationReturnOptions(input_text=False, input_tokens=True),
    ),
    moderations=ModerationParameters(
        # Threshold is set to very low level to flag everything (testing purposes)
        # or set to True to enable HAP with default settings
        hap=ModerationHAP(input=ModerationHAPInput(enabled=True, threshold=0.01))
    ),
)

print(heading("Start conversation with langchain"))
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
    verbose=True
)
while True:
    prompt = input("Enter your prompt (or type 'exit' to end): ")
    if prompt.lower() == 'exit':
        break

    print(f"Request: {prompt}")
    result = conversation.generate(
        messages=[
            [
                SystemMessage(
                    content="""You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature. If a question does not make
any sense, or is not factually coherent, explain why instead of answering something incorrectly.
If you don't know the answer to a question, please don't share false information.
""",
                ),
                HumanMessage(content=prompt),
            ]
        ],
    )
    conversation_id = result.generations[0][0].generation_info["meta"]["conversation_id"]
    print(f"New conversation with ID '{conversation_id}' has been created!")
    print(f"Response: {result.generations[0][0].text}")
    print(result.llm_output)
    print(result.generations[0][0].generation_info)

print(heading("Conversation with langchain ended."))


====================== Start conversation with langchain =======================

Request: Hi I am Saima, I like to eat Biryani


TypeError: LLMChain.generate() got an unexpected keyword argument 'messages'